In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.integrate import odeint #常微分方程式
from scipy.integrate import solve_ivp #常微分方程式
#from scipy.optimize import minimize #非線形計画問題

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.utils.data as utils

import math
import numbers
import warnings
import os
import random

In [ ]:
#original module
import sys
sys.path.append("G:\マイドライブ\python_functions") #win
sys.path.append("/Volumes/GoogleDrive/マイドライブ/python_functions") #mac
sys.path.append("/home/tatematsudaiki/GoogleDrive/python_functions/") #ubuntu
#import
import functions_processing
import functions_time
import functions_analyze_TS
import functions_statistics
import functions_clustering
import functions_plot

In [ ]:
#演算を行うデバイスを設定する　model = model.to(device) / x = x.to(device) で利用
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

print(f'using: {device}')
print(f'number of devices: {torch.cuda.device_count()}')

In [ ]:
#preprocessingの関数

#--------------------------------------------------------------------------------------------------------------------------
#指定したランダムなデータを作成
def return_randam_data(batch_size, list_randam, Standardization_mean, Standardization_std):
    #data to return
    randam_data = torch.Tensor()
    #make
    for _,format in enumerate(list_randam):
        #select
        if format == "float":
            randam_data_now = torch.randn((batch_size,1), dtype=torch.float32)
        elif isinstance(format,tuple):
            randam_data_now = torch.randint(format[0], format[1]+1, (batch_size,1), dtype=torch.float32)
        else:
            print("error!!!!!")
        #add
        randam_data = torch.cat((randam_data,randam_data_now),1)
    #Standardization
    if (Standardization_mean != False) and (Standardization_std != False):
        randam_data = (randam_data - Standardization_mean)/Standardization_std
    #return
    return randam_data

#--------------------------------------------------------------------------------------------------------------------------
#0or1の値 & 一部のみ被らないようにする
def return_0or1_data(batch_size, metadatas_size, list_samedatas_tuple):
    #randome data raw
    randam_data = torch.randint(0, 2, (batch_size,metadatas_size), dtype=torch.float32) #全て0or1
    #change
    for _,tuple in enumerate(list_samedatas_tuple):
        #全て0にする
        randam_data[:, tuple[0]:tuple[1]+1] = 0 #tau2として利用
        if len(tuple)==2:
            #一つだけ1にする
            for bs in range(batch_size):
                random_int = random.randint(tuple[0],tuple[1]+1)
                randam_data[bs, random_int] = 1
        if len(tuple)==3:
            #一つだけ1にする
            randam_data[:, tuple[0]+tuple[2]] = 1
    #return
    return randam_data

#--------------------------------------------------------------------------------------------------------------------------
#maskから一部欠損させる
def lose_value(mask, n):
    #
    for _ in range(n):
        #batch size / mask size
        batch_size = mask.size()[0]
        mask_size = mask.size()[1]
        #lose value
        mask[random.randint(0, batch_size-1)][random.randint(0, mask_size-1)] = 0
    #return
    return mask


#--------------------------------------------------------------------------------------------------------------------------
#「入力時系列データx」から「マスクm」を作成 
# (=「値==0 → 0」「値!=0 → 1」にする)
def make_mask(timeSeries,NA_value): #=torch.Size([timeSeries_size])
    mask = timeSeries.clone().detach() #独立したテンソルを作成
    mask = torch.where(mask==NA_value,0,1).to(torch.float32) #「値==0 → 0」「値!=0 → 1」にする
    return mask #=torch.Size([timeSeries_size])



#--------------------------------------------------------------------------------------------------------------------------
#「マスクm」から「δ」を作成 (等間隔)
def make_delta_equalInterval(mask): #=torch.Size([batch_size, timeSeries_size])
    #値が全て0のδを作成
    delta = torch.zeros(mask.size()).to(device)
    #batch_size, timeSeries_sizeを取得
    batch_size = mask.size()[0]
    timeSeries_size = mask.size()[1]
    #δに値を入れていく
    for batch_index in range(batch_size):
        for timeSeries_index in range(1,timeSeries_size):
            mask_value_before1 = mask[batch_index][timeSeries_index-1]
            if mask_value_before1 == 0:
                delta[batch_index][timeSeries_index] = 1 + delta[batch_index][timeSeries_index-1]
            elif mask_value_before1 == 1:
                delta[batch_index][timeSeries_index] = 1
    #return
    return delta #=torch.Size([batch_size, timeSeries_size])

#「マスクm」から「δ」を作成 (不等間隔)
def make_delta_notEqualInterval(mask,interval): #=torch.Size([batch_size, timeSeries_size])
    #値が全て0のδを作成
    delta = torch.zeros(mask.size()).to(device)
    #batch_size, timeSeries_sizeを取得
    batch_size = mask.size()[0]
    timeSeries_size = mask.size()[1]
    #δに値を入れていく
    for batch_index in range(batch_size):
        for timeSeries_index in range(1,timeSeries_size):
            mask_value_before1 = mask[batch_index][timeSeries_index-1]
            interval_from_before1 = interval[batch_index][timeSeries_index] - interval[batch_index][timeSeries_index-1]
            if mask_value_before1 == 0:
                delta[batch_index][timeSeries_index] = interval_from_before1 + delta[batch_index][timeSeries_index-1]
            elif mask_value_before1 == 1:
                delta[batch_index][timeSeries_index] = interval_from_before1
    #return
    return delta #=torch.Size([batch_size, timeSeries_size])


#--------------------------------------------------------------------------------------------------------------------------
#常微分方程式を定義 / vaccination2 ver
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html
def func(t, M1_M2_A, H2, m, tau1, tau2, d, mu, H1, eta1, K, eta2):
    #print(t)
    #extract
    M1,M2,A = M1_M2_A
    #formulas
    dM1dt = -d*M1 if tau1<=t else 0
    dM2dt = -d*M2 if tau2<=t else 0
    if 0<=t<tau1+eta1:
        dAdt = 0
    elif tau1+eta1<=t<tau2+eta2:
        dAdt = H1*(M1**m/(K**m+M1**m)) - mu*A
    elif tau2+eta2<=t:
        dAdt = H2*((M1+M2)**m/(K**m+(M1+M2)**m)) - mu*A

    #return
    return [dM1dt, dM2dt, dAdt]

#「生成したparamsから作った時系列データ」と「生成した時系列データ」とのlossを計算
def return_timeSeries_of_m_i_loss(params,      #=numpy.ndarray([batch_size,params_size]) #H2, m, tau2
                                batch_size,timeSeries_size):

    #timeSeriesOf_A_generatedByParamsを作成
    timeSeriesOf_A_generatedByParams_ndarray = np.zeros((batch_size,timeSeries_size))
    
    #timeSeriesOf_A_generatedByParamsに値を追加
    for batch_index in range(batch_size):
        #paramsを取り出す(H2,m,tau2)
        H2 = params[batch_index][0]
        m = params[batch_index][1] #
        tau2 = params[batch_index][2] #0<tau2
        #constants (estimated by 12 health care workers)
        M10_M20_A0 = [100,100,0] #M10(=D1),M20(=D2)はテキトーに決められている
        tau1 = 0
        d = 0.693
        mu = 0.885346
        H1 = 975.297
        eta1 = 12.52
        K = 33900
        eta2 = 4.25

        #ODE
        ODE_output = solve_ivp(fun=func,
                                t_span=[0.0,timeSeries_size],
                                y0=M10_M20_A0,
                                method='RK45',
                                t_eval=np.arange(0.0, timeSeries_size, 1),
                                args=(H2, m, tau1, tau2, d, mu, H1, eta1, K, eta2))
        #print(ODE_output.y.shape)

        #check ODE success
        if ODE_output.success == False:
            timeSeriesOf_A_generatedByParams_ndarray[batch_index] = [np.nan for i in range(timeSeries_size)]
            print("ODE error")
            #break
        #pass check
        else:
            #追加
            timeSeriesOf_A_generatedByParams_ndarray[batch_index] = ODE_output.y[2]

    #loss
    #numpy.ndarray -> torch.Tensor
    #timeSeriesOf_V_generatedByParams = torch.tensor(timeSeriesOf_A_generatedByParams_ndarray, dtype=torch.float32).to(device) #=torch.Size([batch_size,timeSeries_size])

    #return
    return timeSeriesOf_A_generatedByParams_ndarray #=torch.Size([batch_size,timeSeries_size])

In [ ]:
#return_0or1_data(batch_size=10, metadatas_size=10, list_samedatas_tuple=[(5,8,1)])

In [ ]:
"""#「生成したparamsから作った時系列データ」と「生成した時系列データ」とのlossを計算
def func_m_i_loss(params,      #=numpy.ndarray([batch_size,params_size]) #H2, m, tau1, tau2
                  timeSeriesOf_A_generatedByGAN): #=torch.Size([batch_size,timeSeries_size])

    #batch_size
    batch_size = timeSeriesOf_A_generatedByGAN.size()[0]
    #timeSeries_size
    timeSeries_size = timeSeriesOf_A_generatedByGAN.size()[1]

    #timeSeriesOf_A_generatedByParamsを作成
    timeSeriesOf_A_generatedByParams_ndarray = np.zeros((batch_size,timeSeries_size))
    
    #timeSeriesOf_A_generatedByParamsに値を追加
    for batch_index in range(batch_size):
        
        #paramsを取り出す(H2,m,tau2)
        H2 = params[batch_index][0]
        m = params[batch_index][1] #
        tau2 = params[batch_index][2] #0<tau2
        #constants (estimated by 12 health care workers)
        M10_M20_A0 = [100,100,0] #M10(=D1),M20(=D2)はテキトーに決められている
        tau1 = 0
        d = 0.693
        mu = 0.885346
        H1 = 975.297
        eta1 = 12.52
        K = 33900
        eta2 = 4.25

        #ODE
        ODE_output = solve_ivp(fun=func,
                                t_span=[0.0,timeSeries_size],
                                y0=M10_M20_A0,
                                method='RK45',
                                t_eval=np.arange(0.0, timeSeries_size, 1),
                                args=(H2, m, tau1, tau2, d, mu, H1, eta1, K, eta2))
        #print(ODE_output.y.shape)
        
        #check params
        if tau2 <= 0:
            timeSeriesOf_A_generatedByParams_ndarray[batch_index] = [100 for i in range(timeSeries_size)]
            print("params error")
            #break
        #check ODE success
        elif ODE_output.success == False:
            timeSeriesOf_A_generatedByParams_ndarray[batch_index] = [100 for i in range(timeSeries_size)]
            print("ODE error")
            #break
        #pass check
        else:
            #追加
            timeSeriesOf_A_generatedByParams_ndarray[batch_index] = ODE_output.y[2]

    #loss
    #numpy.ndarray -> torch.Tensor
    timeSeriesOf_V_generatedByParams = torch.tensor(timeSeriesOf_A_generatedByParams_ndarray, dtype=torch.float32).to(device) #=torch.Size([batch_size,timeSeries_size])
    print(timeSeriesOf_V_generatedByParams)
    #lossを返す
    loss = torch.mean(torch.square(timeSeriesOf_A_generatedByGAN-timeSeriesOf_V_generatedByParams),-1, True) #2乗して平均 #=torch.Size([batch_size,1])
    print(loss)

    #return
    return loss #=torch.Size([batch_size,1])"""

In [ ]:
"""#「生成したparamsから作った時系列データ」と「生成した時系列データ」とのlossを計算
def func_m_i_loss(params,      #=numpy.ndarray([batch_size,params_size]) #H2, m, tau1, tau2
                  timeSeriesOf_A_generatedByGAN): #=torch.Size([batch_size,timeSeries_size])

    #batch_size
    batch_size = timeSeriesOf_A_generatedByGAN.size()[0]
    #timeSeries_size
    timeSeries_size = timeSeriesOf_A_generatedByGAN.size()[1]

    #timeSeriesOf_A_generatedByParamsを作成
    timeSeriesOf_A_generatedByParams_ndarray = np.zeros((batch_size,timeSeries_size))
    
    #timeSeriesOf_A_generatedByParamsに値を追加
    for batch_index in range(batch_size):
        
        #paramsを取り出す(H2,m,tau2)
        H2 = params[batch_index][0]
        m = params[batch_index][1] #
        tau2 = params[batch_index][2] #0<tau2
        #constants (estimated by 12 health care workers)
        M10_M20_A0 = [100,100,0] #M10(=D1),M20(=D2)はテキトーに決められている
        tau1 = 0
        d = 0.693
        mu = 0.885346
        H1 = 975.297
        eta1 = 12.52
        K = 33900
        eta2 = 4.25

        #ODE
        ODE_output = odeint(func=func, #微分方程式
                            y0=M10_M20_A0, #初期値
                            t=np.arange(0, 366, 0.01), #変数
                            args=(H2, m, tau1, tau2, d, mu, H1, eta1, K, eta2)) #定数
    
        #print(ODE_output.y.shape)
        
        #追加
        timeSeriesOf_A_generatedByParams_ndarray[batch_index] = ODE_output[::100][:,2]

    #loss
    #numpy.ndarray -> torch.Tensor
    timeSeriesOf_V_generatedByParams = torch.tensor(timeSeriesOf_A_generatedByParams_ndarray, dtype=torch.float32).to(device) #=torch.Size([batch_size,timeSeries_size])
    #print(timeSeriesOf_V_generatedByParams)
    #lossを返す
    loss = torch.mean(torch.square(timeSeriesOf_A_generatedByGAN-timeSeriesOf_V_generatedByParams),-1, True) #2乗して平均 #=torch.Size([batch_size,1])
    #print(loss)

    #return
    return loss #=torch.Size([batch_size,1])"""

In [ ]:
############################################################################
#[モデル定義]

In [ ]:
"""#GRUD hiddenなし
class GRUD_inputSize1(torch.nn.Module):
    #################################################################
    #[利用する関数]

    #################################################################
    
    def __init__(self, input_size, hidden_size):
        super(GRUD_inputSize1, self).__init__()

        #params------------------------------------------------------
        self.input_size = 1 #入力xの次元数
        #self.input_size = input_size #入力xの次元数
        self.hidden_size = hidden_size #隠れ層hの次元数
        #self.hidden_size = output_size #隠れ層hの次元数
        #self.output_size = input_size #出力層yの次元数

        #self.delta_size = input_size
        #self.mask_size = input_size

        #重みw,バイアスb-----------------------------------------------
        self.gamma_x_l = nn.Linear(self.input_size,self.input_size)
        self.gamma_h_l = nn.Linear(self.input_size,self.input_size)

        self.z_l = nn.Linear(self.input_size+self.input_size+self.input_size, self.input_size)
        self.r_l = nn.Linear(self.input_size+self.input_size+self.input_size, self.input_size)
        self.h_l = nn.Linear(self.input_size+self.input_size+self.input_size, self.input_size)
        
    #################################################################
    #[利用する関数]
    
    #################################################################
    def forward(self, 
                timeSeries_input, #時系列データ =torch.Size([batch_size, timeSeries_size])
                mask_input, #マスクm =torch.Size([batch_size, timeSeries_size])
                delta_input): #δ =torch.Size([batch_size, timeSeries_size])
                
        #-------------------------------------------------------------------------------------------------------
        #[パラメータを取り出す]
        #batch_size :バッチサイズ
        batch_size = timeSeries_input.size(0) #int
        #input_size
        input_size = 1
        #step_size :step_size=時系列データのsize
        step_size = timeSeries_input.size(1) #int
        #hidden_size :隠れ層hのsize
        #hidden_size = self.hidden_size #int
        #output_size = self.input_size #出力層yの次元数
        
        #-------------------------------------------------------------------------------------------------------
        #[処理]

        #比較に利用する零行列
        zeros_usedByGamma_x = torch.zeros(batch_size,self.input_size, dtype=torch.float32).to(device) #=torch.Size([batch_size,input_size]) 
        zeros_usedByGamma_h = torch.zeros(batch_size,self.input_size, dtype=torch.float32).to(device) #=torch.Size([batch_size,input_size])  

        #output :作成する「欠損なし時系列データ」(=yの歴代まとめ)
        timeSeries_output = torch.Tensor().to(device) #=torch.Size([batch_size, timeSeries_size])

        #h :隠れ層h
        #h = torch.zeros(batch_size, input_size, dtype=torch.float32).to(device) #=torch.Size([batch_size,input_size])
        h = torch.randn((batch_size,input_size), dtype=torch.float32).to(device) #=torch.Size([batch_size,input_size])

        #x_last_obsv :直近最後に存在したxの値
        x_last_obsv = timeSeries_input[:,0:1] #=torch.Size([batch_size, input_size])
        #print(x_last_obsv)

        #x_mean :xの平均
        x_mean = torch.sum(torch.where(mask_input==0,mask_input,timeSeries_input).to(torch.float32), 1, keepdim=True) / torch.sum(mask_input, 1, keepdim=True) #=torch.Size([batch_size, input_size])
        #print(x_mean)

        #[処理]
        #print(timeSeries_input)
        #print(mask_input)
        for step_index in range(step_size): #timeSeries_size回実行する
            #timeSeries,m,δから今回に処理する部分を取り出す
            x = timeSeries_input[:,step_index:step_index+1] #=torch.Size([batch_size, input_size])
            m = mask_input[:,step_index:step_index+1] #=torch.Size([batch_size, input_size])
            d = delta_input[:,step_index:step_index+1] #=torch.Size([batch_size, input_size])
            #print(x.size())
            #print(x)
            #print(m)
            #print(d)

            #[x^の作成]---------------------------------------------------------------------------------
            #γ_x (=xのdecay rates)を作成
            gamma_x = torch.exp(-torch.max(zeros_usedByGamma_x, self.gamma_x_l(d))) #=torch.Size([batch_size, input_size])
            #print(gamma_x)

            #x_last_obsvを更新
            x_last_obsv = torch.where(m==1, x, x_last_obsv).to(torch.float32) #=torch.Size([batch_size, input_size])
            #print(x_last_obsv)

            #x^を作成
            x_ = m*x+(1-m)*(gamma_x*x_last_obsv+(1-gamma_x)*x_mean) #=torch.Size([batch_size,input_size])
            #print(x_)

            #[h^の作成]---------------------------------------------------------------------------------
            #γ_h (=hのdecay rates)を作成
            gamma_h = torch.exp(-torch.max(zeros_usedByGamma_h, self.gamma_h_l(d))) #=torch.Size([batch_size,input_size])  
            #print(gamma_h)
            
            #h^を作成
            h_ = gamma_h*h #=torch.Size([batch_size,input_size])
            #print(h_)

            #[GRU部分]---------------------------------------------------------------------------------
            #結合する
            combined = torch.cat((x_, h_, m), 1)

            #z
            z = torch.sigmoid(self.z_l(combined)) #=torch.Size([batch_size,input_size])
            #print(z)

            #r
            r = torch.sigmoid(self.r_l(combined)) #=torch.Size([batch_size,input_size])
            #print(r)

            #結合する
            combined_r = torch.cat((x_, r*h_, m), 1)

            #h_tilde
            h_tilde = torch.tanh(self.h_l(combined_r)) #=torch.Size([batch_size,input_size])
            #print(h_tilde)

            #hを作成
            h = (1-z)*h_ + z*h_tilde #=torch.Size([batch_size,input_size])
            #h = self.layer1(h) #=torch.Size([batch_size,input_size])
            #h = h.unsqueeze(-1) #=torch.Size([batch_size,input_size,1])
            #print(h)

            #y(output)
            #y = m*x+(1-m)*h #=torch.Size([batch_size,input_size])

            #yの歴代まとめ(=timeSeries_output)
            timeSeries_output = torch.cat((timeSeries_output,h),1) #=torch.Size([batch_size,1+1+...]) -> torch.Size([batch_size,timeSeries_size]) 

        #return
        return timeSeries_output #=torch.Size([batch_size,timeSeries_size])"""

In [ ]:
"""#GRUD hiddenあり
class GRUD_inputSize1(torch.nn.Module):
    #################################################################
    #[利用する関数]

    #################################################################
    
    def __init__(self, input_size, hidden_size):
        super(GRUD_inputSize1, self).__init__()

        #params------------------------------------------------------
        self.input_size = 1 #入力xの次元数
        #self.input_size = input_size #入力xの次元数
        self.hidden_size = hidden_size #隠れ層hの次元数
        #self.hidden_size = output_size #隠れ層hの次元数
        #self.output_size = input_size #出力層yの次元数

        #self.delta_size = input_size
        #self.mask_size = input_size

        self.hidden_size_inner = math.ceil(self.hidden_size/2)

        #重みw,バイアスb-----------------------------------------------
        self.gamma_x_l = nn.Linear(self.input_size,self.input_size)
        self.gamma_h_l = nn.Linear(self.input_size,self.hidden_size)

        self.z_l = nn.Linear(self.hidden_size+self.input_size+self.input_size, self.hidden_size)
        self.r_l = nn.Linear(self.hidden_size+self.input_size+self.input_size, self.hidden_size)
        self.h_l = nn.Linear(self.hidden_size+self.input_size+self.input_size, self.hidden_size)
        
        #layer1 :batch_size,hidden_size → batch_size,input_size
        self.layer1 = nn.Sequential(
            nn.BatchNorm1d(self.hidden_size),
            nn.Linear(self.hidden_size, self.hidden_size_inner),
            #nn.BatchNorm1d(self.hidden_size_inner),
            nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(self.hidden_size_inner, self.hidden_size_inner),
            #nn.BatchNorm1d(self.hidden_size_inner),
            #nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(self.hidden_size_inner, self.input_size),
            nn.LeakyReLU(0.2, inplace=True)
        )
        
    #################################################################
    #[利用する関数]
    
    #################################################################
    def forward(self, 
                timeSeries_input, #時系列データ =torch.Size([batch_size, timeSeries_size])
                mask_input, #マスクm =torch.Size([batch_size, timeSeries_size])
                delta_input): #δ =torch.Size([batch_size, timeSeries_size])
                
        #-------------------------------------------------------------------------------------------------------
        #[パラメータを取り出す]
        #batch_size :バッチサイズ
        batch_size = timeSeries_input.size(0) #int
        #input_size
        input_size = 1
        #step_size :step_size=時系列データのsize
        step_size = timeSeries_input.size(1) #int
        #hidden_size :隠れ層hのsize
        hidden_size = self.hidden_size #int
        #output_size = self.input_size #出力層yの次元数
        
        #-------------------------------------------------------------------------------------------------------
        #[処理]

        #比較に利用する零行列
        zeros_usedByGamma_x = torch.zeros(batch_size,self.input_size, dtype=torch.float32).to(device) #=torch.Size([batch_size,input_size]) 
        zeros_usedByGamma_h = torch.zeros(batch_size,self.hidden_size, dtype=torch.float32).to(device) #=torch.Size([batch_size,input_size])  

        #output :作成する「欠損なし時系列データ」(=yの歴代まとめ)
        timeSeries_output = torch.Tensor().to(device) #=torch.Size([batch_size, timeSeries_size])

        #h :隠れ層h
        h = torch.zeros(batch_size, hidden_size, dtype=torch.float32).to(device) #=torch.Size([batch_size,hidden_size])
        #h = torch.randn((batch_size, hidden_size), dtype=torch.float32).to(device) #=torch.Size([batch_size,input_size])

        #x_last_obsv :直近最後に存在したxの値
        x_last_obsv = timeSeries_input[:,0:1] #=torch.Size([batch_size, input_size])
        #print(x_last_obsv)

        #x_mean :xの平均
        x_mean = torch.sum(torch.where(mask_input==0,mask_input,timeSeries_input).to(torch.float32), 1, keepdim=True) / torch.sum(mask_input, 1, keepdim=True) #=torch.Size([batch_size, input_size])
        #print(x_mean)

        #[処理]
        #print(timeSeries_input)
        #print(mask_input)
        for step_index in range(step_size): #timeSeries_size回実行する
            #timeSeries,m,δから今回に処理する部分を取り出す
            x = timeSeries_input[:,step_index:step_index+1] #=torch.Size([batch_size, input_size])
            m = mask_input[:,step_index:step_index+1] #=torch.Size([batch_size, input_size])
            d = delta_input[:,step_index:step_index+1] #=torch.Size([batch_size, input_size])
            #print(x.size())
            #print(x)
            #print(m)
            #print(d)

            #[x^の作成]---------------------------------------------------------------------------------
            #γ_x (=xのdecay rates)を作成
            gamma_x = torch.exp(-torch.max(zeros_usedByGamma_x, self.gamma_x_l(d))) #=torch.Size([batch_size, input_size])
            #print(gamma_x)

            #x_last_obsvを更新
            x_last_obsv = torch.where(m==1, x, x_last_obsv).to(torch.float32) #=torch.Size([batch_size, input_size])
            #print(x_last_obsv)

            #x^を作成
            x_ = m*x+(1-m)*(gamma_x*x_last_obsv+(1-gamma_x)*x_mean) #=torch.Size([batch_size,input_size])
            #print(x_)

            #[h^の作成]---------------------------------------------------------------------------------
            #γ_h (=hのdecay rates)を作成
            gamma_h = torch.exp(-torch.max(zeros_usedByGamma_h, self.gamma_h_l(d))) #=torch.Size([batch_size,hidden_size])  
            #print(gamma_h)
            
            #h^を作成
            h_ = gamma_h*h #=torch.Size([batch_size,hidden_size])
            #print(h_)

            #[GRU部分]---------------------------------------------------------------------------------
            #結合する
            combined = torch.cat((x_, h_, m), 1)

            #z
            z = torch.sigmoid(self.z_l(combined)) #=torch.Size([batch_size,hidden_size])
            #print(z)

            #r
            r = torch.sigmoid(self.r_l(combined)) #=torch.Size([batch_size,hidden_size])
            #print(r)

            #結合する
            combined_r = torch.cat((x_, r*h_, m), 1)

            #h_tilde
            h_tilde = torch.tanh(self.h_l(combined_r)) #=torch.Size([batch_size,hidden_size])
            #print(h_tilde)

            #hを作成
            h = (1-z)*h_ + z*h_tilde #=torch.Size([batch_size,hidden_size])
            #h = h.unsqueeze(-1) #=torch.Size([batch_size,input_size,1])
            #print(h)

            #y(output)
            y = self.layer1(h) #=torch.Size([batch_size,input_size])
            #output = torch.where(m==1, x, y).to(torch.float32)
            #y = m*x+(1-m)*y #=torch.Size([batch_size,input_size])
            #print(y)

            #yの歴代まとめ(=timeSeries_output)
            timeSeries_output = torch.cat((timeSeries_output,y),1) #=torch.Size([batch_size,1+1+...]) -> torch.Size([batch_size,timeSeries_size]) 

        #return
        return timeSeries_output #=torch.Size([batch_size,timeSeries_size])"""

In [ ]:
#Generator(生成器)
class Generator(nn.Module):
    def __init__(self, latent_size, timeSeries_size, params_size, features_size, input_size, GRUD_hidden_size, GRUD_output_size, batch_size):
        super(Generator, self).__init__()
        
        #params
        self.latent_size = latent_size           #Generatorの入力変数(=潜在変数)のsize
        
        self.timeSeries_size = timeSeries_size   #時系列データ部分のsize
        self.params_size = params_size             #パラメータ部分のsize (V0,β,γ,δ)
        self.features_size = features_size      #features
        self.input_size=input_size               #時系列データの数

        self.GRUD_hidden_size=GRUD_hidden_size
        self.GRUD_output_size=GRUD_output_size

        self.batch_size=batch_size

        self.inner_size=10

        #layer0 :latent_size + features_size → param_size
        self.layer0 = nn.Sequential(
            nn.Conv1d(in_channels=1, #入力のチャネル数
                        out_channels=10, #畳み込みをした後のチャネル数。フィルター数。
                        kernel_size=2, #カーネルの大きさ。
                        stride=1, #カーネルをどのくらい移動させるか。
                        padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.LeakyReLU(negative_slope=0.2, inplace=True)
            #nn.Conv1d(in_channels=10, #入力のチャネル数
            #            out_channels=10, #畳み込みをした後のチャネル数。フィルター数。
            #            kernel_size=2, #カーネルの大きさ。
            #            stride=1, #カーネルをどのくらい移動させるか。
            #            padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            #nn.MaxPool1d(kernel_size=2, stride=2),
            #nn.LeakyReLU(negative_slope=0.2, inplace=True),
        )
        
        #layer1 :latent_size + features_size → param_size
        self.layer1 = nn.Sequential(
            #nn.BatchNorm1d(10*10),
            nn.Linear(10*77, 10),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Linear(10, 5),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            #nn.Dropout(p=0.3, inplace=False),       
            nn.Linear(5, self.params_size)
        )

    def forward(self, 
                latent_input, #=torch.Size([batch_size, latent_size])
                features_input): #=torch.Size([batch_size, features_size])
        #layer0のfp
        input0 = torch.cat([latent_input, features_input],1).reshape(self.batch_size, -1, self.latent_size+self.features_size) #(バッチサイズ,チャネル数,データ数)を入力
        #print(input0.size())
        output0 = self.layer0(input0).reshape(-1, 10*77) #=torch.Size([batch_size, param_size])
        #print(output0.size())
        #layer1のfp
        params_output1 = self.layer1(output0) #=torch.Size([batch_size, param_size])

        #return
        return params_output1 #=torch.Size([batch_size, param_size])

In [ ]:
#Discriminator(識別器)
class Discriminator(nn.Module):
    def __init__(self, latent_size, timeSeries_size, params_size, features_size, input_size, GRUD_hidden_size, GRUD_output_size, batch_size):
        super(Discriminator, self).__init__()

        #params
        self.timeSeries_size = timeSeries_size #時系列データ部分のsize
        self.params_size = params_size           #パラメータ部分のsize (V0,β,γ,δ)
        self.features_size=features_size      #features
        self.input_size=input_size             #時系列データの数
        latent_size=latent_size                #Generatorの入力変数(=潜在変数)のsize

        self.GRUD_hidden_size=GRUD_hidden_size
        self.GRUD_output_size=GRUD_output_size

        self.batch_size=batch_size

        self.inner_size = 1

        #layer0 :params,features  → validity(0~1の値)
        self.layer0 = nn.Sequential(
            nn.BatchNorm1d(self.params_size+self.features_size),
            nn.Linear(self.params_size+self.features_size, 10),
            #nn.BatchNorm1d(3),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(10,5),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Linear(5,1),
            nn.Sigmoid()
        )

        """#layer3 :params → inner_size
        self.layer3 = nn.Sequential(
            nn.BatchNorm1d(self.params_size),
            nn.Linear(self.params_size, self.inner_size),
            #nn.BatchNorm1d(2),
            #nn.LeakyReLU(negative_slope=0.2, inplace=True),
            #nn.Linear(2, self.inner_size),
        )"""

        """#layer4 :features → inner_size
        self.layer4 = nn.Sequential(
            nn.BatchNorm1d(self.features_size),
            nn.Linear(self.features_size, self.inner_size),
            #nn.BatchNorm1d(3),
            #nn.LeakyReLU(negative_slope=0.2, inplace=True),
            #nn.Linear(3, self.inner_size),
        )"""

        """#layer5 :(layer2のinner_size,layer3のinner_size) → validity(0~1の値)
        self.layer5 = nn.Sequential(
            #nn.BatchNorm1d(self.inner_size*3),
            nn.Linear(self.inner_size*2, 1),
            #nn.BatchNorm1d(3),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(10, 1),
            nn.Sigmoid()
        )"""

    def forward(self,
                params_input, #=torch.Size([batch_size, param_size])
                features_input): #=torch.Size([batch_size, features_size])

        #layer3のfp :params_size → inner_size
        #validity_output3 = self.layer3(params_input) #=torch.Size([batch_size, 1])

        #layer4のfp :features_size → inner_size
        #validity_output4 = self.layer4(features_input) #=torch.Size([batch_size, 1])

        #layer5のfp :latent_size → size1(0～1の値)
        #validity_output5 = self.layer5(torch.cat([validity_output3, validity_output4],1)) #=torch.Size([batch_size, 1])

        #layer0のfp :params_size + features_size → size1(0～1の値)
        validity_output0 = self.layer0(torch.cat([params_input, features_input],1)) #=torch.Size([batch_size, 1])
        #validity_output0 = self.layer0(params_input) #=torch.Size([batch_size, 1])

        return validity_output0 #=torch.Size([batch_size, 1])

In [ ]:
############################################################################
#[設定値]
input_size = 1       #時系列データの数(変数の数)
#設定値(GRUD)
GRUD_hidden_netG_size = 10            #GRUDのhの次元数
GRUD_hidden_netD_size = 2            #GRUDのhの次元数
GRUD_output_size = input_size     #GRUDのyの次元数

#設定値(DataLoader)
batch_size = 300 #バッチサイズ
dataloader_suffle = True #dataloaderをシャッフル
dataloader_drop_last=True #バッチサイズで取り出した時の余りを除去

#設定値(GAN学習)
EPOCHS = 500000 #エポック数
latent_size = 10 #Generator(生成器)の入力値(=潜在変数)の次元数

netG_learn_n_start = 1 #識別器の学習1回につき何回生成器を学習させるか(start)
netG_learn_n_end = 1 #識別器の学習1回につき何回生成器を学習させるか(end)

#設定値(m-i-GAN)
#m_i_GAN_start = 300 #m_i_lossの計算をいつ始めるか(エポック数で指定)

#設定値(評価)
plot_fake_datas_interval = 10000 #生成データの表示間隔(エポック数で指定)
plot_fake_datas_n = 300 #生成データの表示数

plot_losses_interval = plot_fake_datas_interval #loss履歴の表示間隔(エポック数で指定)

print_loss_interval = 100 #lossをprintする間隔(epoch数で指定)

plot_params_interval = plot_fake_datas_interval #paramsをscatter plot
plot_params_n = plot_fake_datas_n

#設定値(保存)
#save_models = 100

In [ ]:
############################################################################
#[データセットの作成]
#read
df_datas = pd.read_csv('../datas/all_fitted_dummyVariable_0or1_withoutgroup0.csv', index_col=0) #補間後の欠損なし時系列データ
#df_datas = pd.read_csv('../features_and_params_and_timeSeries_raw_vaccination2_dummyVariable_withoutgroup0.csv', index_col=0) #補間前の欠損あり時系列データ
print(df_datas)

#outlier
df_datas = functions_statistics.return_df_without_outlier(df_input=df_datas, list_of_row_use=["H2","m"])

#extract
df_timeSeries = df_datas.loc[:,[str(i) for i in range(0,366)]] #補間後の欠損なし時系列データ
df_params = df_datas.loc[:,["H2","m"]]
df_features = df_datas.loc[:,["age_{}".format(i) for i in range(0,100+1)]\
                            +
                            ["bloodtype_a","bloodtype_b","bloodtype_o"]
                            +
                            ["drinking_sometime","drinking_everyday"]
                            +
                            [#"tau2",
                            "sex",
                            "sideReaction1stOr2nd_localPain","sideReaction1stOr2nd_under37.5","sideReaction1stOr2nd_over37.5","sideReaction1stOr2nd_dull","sideReaction1stOr2nd_headache","sideReaction1stOr2nd_jointPain","sideReaction1stOr2nd_diarrhea","sideReaction1stOr2nd_nausea","sideReaction1stOr2nd_dizziness","sideReaction1stOr2nd_others",
                            "smoking",
                            "medicine_steroid","medicine_NSAIDs","medicine_acetaminophen","medicine_antihistamine","medicine_immunosuppression","medicine_biologicalDrag","medicine_anticancer",
                            "disease_covid19Me","disease_highBloodPressure","disease_diabetes","disease_bronchialAsthma","disease_anaphylaxis","disease_mentalIllness","disease_covid19Family","disease_gout","disease_highCholesterol","disease_rheumatism","disease_respiratory","disease_circulatoryOrgan","disease_connectiveTissueDisease","disease_allergies","disease_immunodeficiency","disease_malignantTumor","disease_thyroid","disease_liver","disease_kidney","disease_others",
                            "BCGVaccine"]]
#
#df_features["age"] = df_features["age"]/100

#df_params :column(縦)ごとにNormalize
df_params,mean_params,std_params = functions_statistics.Standardization_row(df=df_params)

mean_params = torch.tensor(np.tile(mean_params,(batch_size,1)), dtype=torch.float32).to(device)
std_params = torch.tensor(np.tile(std_params,(batch_size,1)), dtype=torch.float32).to(device)


#randamデータ生成のためのlist
"""list_to_make_randomData = [#(1,365),
                            (0,1),
                            (0,100),
                            (0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),
                            (0,1),
                            (0,1),
                            (0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),
                            (0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),
                            #(-1,3),
                            (0,1)]
print(len(list_to_make_randomData))"""
list_samedatas_tuple = [(0,100),(101,103),(104,105)]
list_samedatas_tuple_plot_20 = [(0,100,20),(101,103),(104,105)]
list_samedatas_tuple_plot_80 = [(0,100,80),(101,103),(104,105)]


#設定値(データセット)
timeSeries_size = 366 #時系列データのsize
params_size = len(df_params.columns) #パラメータ部分のsize (今回はH2,m)
features_size = len(df_features.columns)
print(timeSeries_size)
print(params_size)
print(features_size)



#DataSetの実装
class DataSet():
    def __init__(self):
        self.params = torch.tensor(df_params.values, dtype=torch.float32) #推定したパラメータ(H2, m)
        self.features = torch.tensor(df_features.values, dtype=torch.float32)

    def __len__(self):
        return len(self.params) # データ数を返す

    def __getitem__(self, index):
        return self.params[index],self.features[index] #index番目の入出力ペアを返す

#DataLoaderの実装
dataset = DataSet() #DataSetクラスのインスタンスを作成
dataloader = torch.utils.data.DataLoader(dataset, #dataset
                                        batch_size=batch_size, #バッチサイズ
                                        shuffle=dataloader_suffle, #dataloaderをシャッフル
                                        drop_last=dataloader_drop_last) #バッチサイズで取り出した時の余りを除去

In [ ]:
"""
###################################################
#plot
for index_int, row_series in df_timeSeries.iterrows():
    #times
    times = df_timeSeries.columns.values
    #extract
    values = row_series
    #plot
    plt.plot(times, values, marker=".", color="black", markersize=0.03, linewidth=0.03, zorder=1)

#----------------------------------------------------------------
#plt.xlim(0,100)
plt.ylim(0,5000)
plt.xticks(np.arange(0, 365, 30))
#show
plt.show()
"""

In [ ]:
############################################################################
#[使う物の作成]
#インスタンス作成 & GPUに送る
netG = Generator(latent_size=latent_size, timeSeries_size=timeSeries_size, params_size=params_size, features_size=features_size, input_size=input_size, GRUD_hidden_size=GRUD_hidden_netG_size, GRUD_output_size=GRUD_output_size, batch_size=batch_size).to(device)
netD = Discriminator(latent_size=latent_size, timeSeries_size=timeSeries_size, params_size=params_size, features_size=features_size, input_size=input_size, GRUD_hidden_size=GRUD_hidden_netD_size, GRUD_output_size=GRUD_output_size, batch_size=batch_size).to(device)
print(sum(p.numel() for p in netG.parameters()))
print(sum(p.numel() for p in netD.parameters()))

#
"""for p in netG.parameters():
    nn.init.uni(p,7.0,10.0)
for p in netD.parameters():
    nn.init.normal_(p,0.0,1.0)"""
"""for p in netG.parameters():
    nn.init.normal_(p, 0.0, 2.0)
for p in netD.parameters():
    nn.init.normal_(p, 0.0, 1.0)"""

# 複数GPU使用宣言
"""if device == "cuda":
    print("multi gpu")
    netG = torch.nn.DataParallel(netG) # make parallel
    netD = torch.nn.DataParallel(netD) # make parallel
    torch.backends.cudnn.benchmark=True
print(device)
"""

#誤差関数
#criterion = nn.BCELoss().to(device) #誤差関数をGPUに送る

#最適化アルゴリズム
lr = 0.0001 #学習率
b1 = 0.5
b2 = 0.999
optimizer_netG = torch.optim.Adam(netG.parameters(), #最適化対象のパラメータ　w <- w-∂E/∂w(E=値)
                                lr=lr, 
                                betas=[b1, b2])
optimizer_netD = torch.optim.Adam(netD.parameters(), 
                                lr=lr, 
                                betas=[b1, b2])

In [ ]:
#別々version & fp複数回ver
############################################################################
#[学習]
losses_real_validity = [] #「全エポックのloss履歴」(real)
losses_fake_validity = [] #「全エポックのloss履歴」(fake)

for epoch in range(1, EPOCHS+1): #エポック数分だけ実行
    loss_real_validity_sum = 0.0 #「今エポックの合計loss」(real)
    loss_fake_validity_sum = 0.0 #「今エポックの合計loss」(fake)
    loss_m_i_loss_sum = 0.0 #「今エポックの合計loss」(m-i-loss)
    
    iter_count = 1
    for real_params,real_features in dataloader:
        #---------------------------------------------------------------------------------------------------------
        #「識別器の学習(real)」
        #real data
        real_params = real_params.to(device)
        real_features = real_features.to(device)

        #最適化対象のすべてのパラメータの勾配を0にする
        optimizer_netD.zero_grad() 
        
        #real dataでnetDをfp 
        real_validity = netD(params_input=real_params, features_input=real_features) #=torch.Size([batch_size, 1])
        
        #loss
        loss_netD = -torch.mean(real_validity)
        
        #bp(=各パラメータの偏微分を計算)
        loss_netD.backward()
        #最適化
        optimizer_netD.step()

        #validity
        real_validity_netD = torch.sigmoid(-loss_netD).item()

        #---------------------------------------------------------------------------------------------------------
        #「識別器の学習(fake)」
        #fake data
        random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
        #random_features = torch.randn((batch_size,features_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
        #random_features = return_randam_data(batch_size, list_to_make_randomData, Standardization_mean=False, Standardization_std=False).to(device)
        random_features = return_0or1_data(batch_size=batch_size, metadatas_size=features_size, list_samedatas_tuple=list_samedatas_tuple).to(device) 
        
        #最適化対象のすべてのパラメータの勾配を0にする
        optimizer_netD.zero_grad() 

        #fake dataでnetGをfp
        fake_params = netG(latent_input=random_latent, features_input=random_features) #=torch.Size([batch_size, timeSeries_size]), torch.Size([batch_size, param_size])
        fake_validity = netD(params_input=fake_params, features_input=random_features) #=torch.Size([batch_size, 1])

        #loss
        loss_netD = torch.mean(fake_validity) #これが小さくなってほしい

        #bp(=各パラメータの偏微分を計算)
        loss_netD.backward()
        #最適化
        optimizer_netD.step()

        #validity
        fake_validity_netD = torch.sigmoid(loss_netD).item()
        
        #---------------------------------------------------------------------------------------------------------
        #「生成器の学習」
        
        #識別器の学習の何回おきに生成器を学習させるか(イテレーション数で指定)
        for n in range(netG_learn_n_start):
        #for n in range(netG_learn_n_start + int((netG_learn_n_end-netG_learn_n_start)*epoch/EPOCHS)):
            #fake data
            random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            #random_features = torch.randn((batch_size,features_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            #random_features = return_randam_data(batch_size, list_to_make_randomData, Standardization_mean=False, Standardization_std=False).to(device)
            random_features = return_0or1_data(batch_size=batch_size, metadatas_size=features_size, list_samedatas_tuple=list_samedatas_tuple).to(device)

            #最適化対象のすべてのパラメータの勾配を0にする
            optimizer_netG.zero_grad() 

            #fake dataでnetG & netDをfp
            fake_params = netG(latent_input=random_latent, features_input=random_features) #=torch.Size([batch_size, param_size])
            fake_validity = netD(params_input=fake_params, features_input=random_features) #=torch.Size([batch_size, 1])
            
            #loss
            loss_netG = -torch.mean(fake_validity) #これが小さくなってほしい #=torch.Size([1])

            #bp(=偏微分を計算)
            loss_netG.backward()
            #最適化
            optimizer_netG.step()

            #validity
            #fake_validity_netG = torch.sigmoid(-loss_netG).item()

        #---------------------------------------------------------------------------------------------------------
        #print
        """if epoch < m_i_GAN_start:
            print("iter:{5} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4}".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, iter_count))
            #print('[Epoch %d/%d] [loss_netD: %f] [loss_netG: %f]'% (epoch, EPOCHS, loss_netD.item(), loss_netG.item()))
        elif m_i_GAN_start <= epoch:
            print("iter:{5} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4}".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, iter_count))
        #print("iter:{5} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:None".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, iter_count))"""

        #---------------------------------------------------------------------------------------------------------
        #sum
        loss_real_validity_sum += real_validity_netD
        loss_fake_validity_sum += fake_validity_netD
        
        #---------------------------------------------------------------------------------------------------------
        #イテレーション数を+1
        iter_count += 1
        #print(iter_count)

    ############################################################################
    #「今エポックのloss平均」を「loss履歴」に追加
    losses_real_validity.append(loss_real_validity_sum/(iter_count-1)) 
    losses_fake_validity.append(loss_fake_validity_sum/(iter_count-1))
    
    ############################################################################
    #[評価]
    #---------------------------------------------------------------------------------------------------------
    #今エポックの、「エポック数」,「識別機のloss」,「生成器のloss」を表示
    if epoch%print_loss_interval == 0:
        print("epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3}".format(epoch, EPOCHS, losses_real_validity[-1], losses_fake_validity[-1]))
    #---------------------------------------------------------------------------------------------------------
    #エポックの、paramsをscatterplot
    if epoch%plot_params_interval == 0:
        #---------------------------------------------------------------------------------------------------------
        plt.figure(figsize=(5, 5))
        #---------------------------------------------------------------------------------------------------------
        #real data
        plt.scatter(df_params["H2"].values*std_params[0,0].item()+mean_params[0,0].item(), df_params["m"].values*std_params[0,1].item()+mean_params[0,1].item(), s=0.2, c="black")
        #---------------------------------------------------------------------------------------------------------
        for _ in range(math.ceil(plot_params_n/batch_size)):
            #fake data
            random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            #random_features = return_randam_data(batch_size, list_to_make_randomData, Standardization_mean=False, Standardization_std=False).to("cpu")

            #---------------------------------------------------------------------------------------------------------
            random_features = return_0or1_data(batch_size=batch_size, metadatas_size=features_size, list_samedatas_tuple=list_samedatas_tuple_plot_20).to(device)
            #tau2,age変更
            #random_features[:, 1] = 20 #age
            #"生成器入力用ランダムデータ"を生成器でfp
            fake_params_Normalized = netG(latent_input=random_latent, features_input=random_features) #=torch.Size([batch_size, param_size])
            #-----------------------------------
            #値を戻す
            fake_params = fake_params_Normalized*std_params + mean_params
            #-----------------------------------
            #scatter plot
            plt.scatter(fake_params[:, 0].detach().numpy(), fake_params[:, 1].detach().numpy(), s=2, c="blue", zorder=10)

            #---------------------------------------------------------------------------------------------------------
            random_features = return_0or1_data(batch_size=batch_size, metadatas_size=features_size, list_samedatas_tuple=list_samedatas_tuple_plot_80).to(device)
            #tau2,age変更
            #random_features[:, 1] = 80 #age
            #"生成器入力用ランダムデータ"を生成器でfp
            fake_params_Normalized = netG(latent_input=random_latent, features_input=random_features) #=torch.Size([batch_size, param_size])
            #-----------------------------------
            #値を戻す
            fake_params = fake_params_Normalized*std_params + mean_params
            #-----------------------------------
            #scatter plot
            plt.scatter(fake_params[:, 0].detach().numpy(), fake_params[:, 1].detach().numpy(), s=2, c="maroon", zorder=11)
            
        #---------------------------------------------------------------------------------------------------------
        plt.grid()
        plt.xlabel("H2")
        plt.ylabel("m")
        plt.show()
    #---------------------------------------------------------------------------------------------------------
    #今エポックの、生成データをplot
    if epoch%plot_fake_datas_interval == 0:
        #matplotlibのfig作成
        fig = plt.figure()
        #time作成
        t = np.arange(0, timeSeries_size, 1)
        #生成時系列データをplot
        for j in range(math.ceil(plot_fake_datas_n/batch_size)):
            #---------------------------------------------------------------------------------------------------------
            #fake data
            random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            #random_features = return_randam_data(batch_size, list_to_make_randomData, Standardization_mean=False, Standardization_std=False).to("cpu")

            #---------------------------------------------------------------------------------------------------------
            random_features = return_0or1_data(batch_size=batch_size, metadatas_size=features_size, list_samedatas_tuple=list_samedatas_tuple_plot_20).to(device)
            #feature変更
            #random_features[:, 1] = 20 #age
            #"生成器入力用ランダムデータ"を生成器でfp
            fake_params_Normalized = netG(latent_input=random_latent, features_input=random_features) #=torch.Size([batch_size, param_size])
            #-----------------------------------
            #値を戻す
            fake_params = fake_params_Normalized*std_params + mean_params
            #h2,m,tau2にする
            random_features[:, 1] = 30 #tau2として利用
            fake_params_to_use_ndarray = torch.cat((fake_params,random_features[:,1:2]),1).detach().numpy()
            #timeSeries化
            fake_timeSeries_ndarray = return_timeSeries_of_m_i_loss(params=fake_params_to_use_ndarray, batch_size=batch_size, timeSeries_size=timeSeries_size)
            #-----------------------------------
            #時系列データ部分をplot
            for b in range(batch_size):
                plt.plot(t, fake_timeSeries_ndarray[b], c='blue', linewidth=0.2)
            #figの見た目設定
            plt.xlabel('day') #x軸ラベル
            plt.ylabel('antibody titer') #y軸ラベル
            plt.title('fake data (z='+str(latent_size)+', %d epochs)' % epoch) #タイトル
            plt.xticks(np.arange(0, 365, 30))
            #pauseする
            plt.pause(.01)
            #図を消す
            plt.close()
            
            #---------------------------------------------------------------------------------------------------------
            random_features = return_0or1_data(batch_size=batch_size, metadatas_size=features_size, list_samedatas_tuple=list_samedatas_tuple_plot_80).to(device)
            #feature変更
            #random_features[:, 1] = 80 #age
            #"生成器入力用ランダムデータ"を生成器でfp
            fake_params_Normalized = netG(latent_input=random_latent, features_input=random_features) #=torch.Size([batch_size, param_size])
            #-----------------------------------
            #値を戻す
            fake_params = fake_params_Normalized*std_params + mean_params
            #h2,m,tau2にする
            random_features[:, 1] = 30 #tau2として利用
            fake_params_to_use_ndarray = torch.cat((fake_params,random_features[:,1:2]),1).detach().numpy()
            #timeSeries化
            fake_timeSeries_ndarray = return_timeSeries_of_m_i_loss(params=fake_params_to_use_ndarray, batch_size=batch_size, timeSeries_size=timeSeries_size)
            #-----------------------------------
            #時系列データ部分をplot
            for b in range(batch_size):
                plt.plot(t, fake_timeSeries_ndarray[b], c='maroon', linewidth=0.2)
            #figの見た目設定
            plt.xlabel('day') #x軸ラベル
            plt.ylabel('antibody titer') #y軸ラベル
            plt.title('fake data (z='+str(latent_size)+', %d epochs)' % epoch) #タイトル
            plt.xticks(np.arange(0, 365, 30))
            #pauseする
            plt.pause(.01)
            #図を消す
            plt.close()
            
    #---------------------------------------------------------------------------------------------------------
    #今エポックまでの、loss履歴をplot
    if epoch%plot_losses_interval == 0:
        #matplotlibのfig作成
        fig2 = plt.figure()
        #realのloss履歴をプロット
        plt.plot(range(len(losses_real_validity)), losses_real_validity, label='real_loss')
        #fakeのloss履歴をプロット
        plt.plot(range(len(losses_fake_validity)), losses_fake_validity, label='fake_loss')
        #figの見た目設定
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.ylim(0,1)
        plt.yticks(np.arange(0, 1, 0.1))
        plt.legend()
        plt.grid()
        #表示
        plt.show()
        #pauseする
        plt.pause(.01)
        #図を消す
        plt.close()
    #---------------------------------------------------------------------------------------------------------
    """
    #今エポックの、modelをsave
    if epoch%save_models == 0:
        #make directory
        dir_path = "/Users/tatematsudaiki/Desktop/VPGAM_models"
        os.makedirs(dir_path,exist_ok=True)
        #save models
        torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(dir_path,"netG",epoch))
        torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(dir_path,"netD",epoch))
    """

In [ ]:
"""#別々version & fp一回のみver
############################################################################
#[学習]
losses_real_validity = [] #「全エポックのloss履歴」(real)
losses_fake_validity = [] #「全エポックのloss履歴」(fake)
losses_m_i_loss = []

for epoch in range(1, EPOCHS+1): #エポック数分だけ実行
    loss_real_validity_sum = 0.0 #「今エポックの合計loss」(real)
    loss_fake_validity_sum = 0.0 #「今エポックの合計loss」(fake)
    loss_m_i_loss_sum = 0.0 #「今エポックの合計loss」(m-i-loss)
    
    iter_count = 1
    for real_timeSeries,real_params,real_features,real_mask in dataloader:
        #---------------------------------------------------------------------------------------------------------
        #[real data]
        real_timeSeries = real_timeSeries.to(device)
        real_params = real_params.to(device)
        real_features = real_features.to(device)
        real_mask = lose_value(real_mask, 3).to(device)
        #---------------------------------------------------------------------------------------------------------
        #[fake data]
        random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
        random_features = torch.randn((batch_size,features_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
        #random_features = return_randam_data(batch_size, list_to_make_randamData, Standardization_mean=False, Standardization_std=False).to(device)
        random_mask = torch.randint(0, 2, (batch_size,timeSeries_size), dtype=torch.float32).to(device)  #値が0or1のtorch.Size([batch_size, timeSeries_size])を作成

        #---------------------------------------------------------------------------------------------------------
        #real dataのfp 
        real_validity = netD(timeSeries_input=real_timeSeries, params_input=real_params, features_input=real_features, mask_input=real_mask) #=torch.Size([batch_size, 1])

        #fake dataのfp
        fake_params, fake_timeSeries = netG(latent_input=random_latent, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, timeSeries_size]), torch.Size([batch_size, param_size])
        fake_validity = netD(timeSeries_input=fake_timeSeries, params_input=fake_params, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, 1])
        
        #---------------------------------------------------------------------------------------------------------
        #「識別器の学習」
        #最適化対象のすべてのパラメータの勾配を0にする
        optimizer_netD.zero_grad()
        
        #loss
        loss_netD = - torch.mean(real_validity) + torch.mean(fake_validity)
        real_validity_netD = torch.sigmoid(torch.mean(real_validity)).item()
        fake_validity_netD = torch.sigmoid(torch.mean(fake_validity)).item()
        
        #bp(=各パラメータの偏微分を計算)
        loss_netD.backward(retain_graph=True)
        #最適化
        optimizer_netD.step()
        
        #---------------------------------------------------------------------------------------------------------
        #「生成器の学習」
        #識別器の学習の何回おきに生成器を学習させるか(イテレーション数で指定)
        for n in range(netG_learn_n_start):
        #for n in range(netG_learn_n_start + int((netG_learn_n_end-netG_learn_n_start)*epoch/EPOCHS)):
            #最適化対象のすべてのパラメータの勾配を0にする
            optimizer_netG.zero_grad()
            
            #lossを計算(「本物は大きく, 偽物は小さく なってほしい」の逆)(=fake_validityは大きくなってほしい)
            loss_netG = -torch.mean(fake_validity) #これが小さくなってほしい #=torch.Size([1])
            fake_validity_netG = torch.sigmoid(-loss_netG).item()

            #bp(=偏微分を計算)
            loss_netG.backward(retain_graph=True)
            #最適化
            optimizer_netG.step()

            #---------------------------------------------------------------------------------------------------------
            #「パラメータに合うように、時系列を学習」
            #100エポック以上になったら「生成したparamsから生成した時系列データ」と「ganから生成した時系列データ」とのlossをloss_netGに追加
            if epoch >= m_i_GAN_start: #100エポック以上になったら
                #最適化対象のすべてのパラメータの勾配を0にする
                optimizer_netG.zero_grad()

                #取り出す & 値を戻す
                params_detached = fake_params.detach()*std_params+mean_params #値は貰って共有、計算グラフは切る #& ndarry化
                features_detached = random_features.detach()*std_features+mean_features #値は貰って共有、計算グラフは切る #& ndarry化

                #m-i-lossに利用するparams
                params_to_use_detached = torch.cat((params_detached,features_detached[:,0:1]),1).to("cpu").numpy()
                #m_i_lossの影響度
                #degreeOfInfluence = 1/1
                #degreeOfInfluence = torch.abs(fake_validity_detached)/10  #多分変

                #m_i_lossを計算 & 値を戻す
                timeSeries_of_m_i_loss = ((return_timeSeries_of_m_i_loss(params=params_to_use_detached, batch_size=batch_size, timeSeries_size=timeSeries_size).to(device) - mean_timeSeries) / std_timeSeries) #=torch.Size([batch_size,timeSeries_size])

                #loss
                m_i_loss = torch.mean(torch.mean(torch.square(fake_timeSeries - timeSeries_of_m_i_loss),-1, True)) #2乗して平均 #=torch.Size([1])
                fake_validity_m_i_loss = torch.sigmoid(m_i_loss).item()

                #bp(=偏微分を計算)
                m_i_loss.backward(retain_graph=True)
                #最適化
                optimizer_netG.step()
            else:
                m_i_loss = torch.tensor(np.nan)

        #---------------------------------------------------------------------------------------------------------
        # 誤差逆伝播を実行後、計算グラフを削除
        #del loss 

        #---------------------------------------------------------------------------------------------------------
        #print
        if epoch < m_i_GAN_start:
            print("iter:{5} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:None".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, iter_count))
            #print('[Epoch %d/%d] [loss_netD: %f] [loss_netG: %f]'% (epoch, EPOCHS, loss_netD.item(), loss_netG.item()))
        elif m_i_GAN_start <= epoch:
            print("iter:{6} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:{5}".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, fake_validity_m_i_loss, iter_count))
        #print("iter:{5} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:None".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, iter_count))

        #---------------------------------------------------------------------------------------------------------
        #sum
        loss_real_validity_sum += real_validity_netD
        loss_fake_validity_sum += fake_validity_netD
        loss_m_i_loss_sum += fake_validity_m_i_loss
        
        #---------------------------------------------------------------------------------------------------------
        #イテレーション数を+1
        iter_count += 1
        #print(iter_count)

    ############################################################################
    #「今エポックの最後のイテレーションのloss」を「loss履歴」に追加
    #losses_real_validity.append(real_validity_netD) 
    #losses_fake_validity.append(fake_validity_netD)
    #losses_m_i_loss.append(round(m_i_loss.item(),30))

    losses_real_validity.append(loss_real_validity_sum/(iter_count-1)) 
    losses_fake_validity.append(loss_fake_validity_sum/(iter_count-1))
    losses_m_i_loss.append(loss_m_i_loss_sum/(iter_count-1))
    
    ############################################################################
    #[評価]
    #---------------------------------------------------------------------------------------------------------
    #指定エポック毎に、「エポック数」,「識別機のloss」,「生成器のloss」を表示
    if epoch%print_loss_interval == 0:
        if epoch < m_i_GAN_start:
            print("epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} m-i-loss:None".format(epoch, EPOCHS, losses_real_validity[-1], losses_fake_validity[-1]))
            #print('[Epoch %d/%d] [loss_netD: %f] [loss_netG: %f]'% (epoch, EPOCHS, loss_netD.item(), loss_netG.item()))
        elif m_i_GAN_start <= epoch:
            print("epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} m-i-loss:{4}".format(epoch, EPOCHS, losses_real_validity[-1], losses_fake_validity[-1], losses_m_i_loss[-1]))
    #---------------------------------------------------------------------------------------------------------
    #指定エポック毎に、生成データをplot
    if epoch%plot_fake_datas_interval == 0:
        #matplotlibのfig作成
        fig = plt.figure()
        #time作成
        t = np.arange(0, timeSeries_size, 1)
        #生成時系列データをplot
        for j in range(math.ceil(plot_fake_datas_n/batch_size)):
            #-----------------------------------
            #[random data]生成器入力用データ
            random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            random_features = return_randam_data(batch_size, list_to_make_randamData, Standardization_mean=False, Standardization_std=False).to(device)
            random_mask = torch.randint(0, 2, (batch_size,timeSeries_size), dtype=torch.float32).to(device)  #値が0or1のtorch.Size([batch_size, timeSeries_size])を作成
            #値を変換
            random_features = random_features*std_features+mean_features
            #tauのみ変更
            random_features[:, 0] = 100
            #-----------------------------------
            #"生成器入力用ランダムデータ"を生成器でfp
            fake_params, fake_timeSeries = netG(latent_input=random_latent, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, timeSeries_size]), torch.Size([batch_size, param_size])
            #値を戻す
            fake_timeSeries = fake_timeSeries*std_timeSeries+mean_timeSeries
            #-----------------------------------
            #matplotlibで使うためにndarray化
            fake_timeSeries_numpy = fake_timeSeries.detach().to("cpu").numpy().copy() #detach() & ndarray化
            #時系列データ部分をplot
            for b in range(batch_size):
                plt.plot(t, fake_timeSeries_numpy[b], c='maroon', linewidth=0.2)
        #figの見た目設定
        plt.xlabel('day') #x軸ラベル
        plt.ylabel('antibody titer') #y軸ラベル
        plt.title('fake data (z='+str(latent_size)+', %d epochs)' % epoch) #タイトル
        plt.xticks(np.arange(0, 365, 30))
        #pauseする
        plt.pause(.01)
        #図を消す
        plt.close()
    #---------------------------------------------------------------------------------------------------------
    #指定エポック毎に、loss履歴をplot
    if epoch%plot_losses_interval == 0:
        #matplotlibのfig作成
        fig2 = plt.figure()
        #realのloss履歴をプロット
        plt.plot(range(len(losses_real_validity)), losses_real_validity, label='real_loss')
        #fakeのloss履歴をプロット
        plt.plot(range(len(losses_fake_validity)), losses_fake_validity, label='fake_loss')
        #figの見た目設定
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.ylim(0,1)
        plt.yticks(np.arange(0, 1, 0.1))
        plt.legend()
        plt.grid()
        #表示
        plt.show()
        #pauseする
        plt.pause(.01)
        #図を消す
        plt.close()
    #---------------------------------------------------------------------------------------------------------
    """
    #modelをsave
    if epoch%save_models == 0:
        #make directory
        dir_path = "/Users/tatematsudaiki/Desktop/VPGAM_models"
        os.makedirs(dir_path,exist_ok=True)
        #save models
        torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(dir_path,"netG",epoch))
        torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(dir_path,"netD",epoch))
    """"""

In [ ]:
"""#ノーマルversion
############################################################################
#[学習]
losses_real_validity = [] #「全エポックのloss履歴」(real)
losses_fake_validity = [] #「全エポックのloss履歴」(fake)
losses_m_i_loss = []

for epoch in range(1, EPOCHS+1): #エポック数分だけ実行
    loss_real_validity_sum = 0.0 #「今エポックの合計loss」(real)
    loss_fake_validity_sum = 0.0 #「今エポックの合計loss」(fake)
    loss_m_i_loss_sum = 0.0 #「今エポックの合計loss」(m-i-loss)
    
    iter_count = 1
    for real_timeSeries,real_params,real_features,real_mask in dataloader:
        #---------------------------------------------------------------------------------------------------------
        #[real data]
        #real_timeSeries = real_timeSeries/1000
        real_timeSeries = real_timeSeries.to(device)
        real_params = real_params.to(device)
        real_features = real_features.to(device)
        real_mask = lose_value(real_mask, 3).to(device)
        #print(real_timeSeries)
        #print(real_params_and_features)
        #print(real_mask)
        
        #---------------------------------------------------------------------------------------------------------
        #「識別器の学習」
        #[random data]生成器入力用データ
        random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
        random_features = torch.randn((batch_size,features_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
        #random_features = return_randam_data(batch_size, list_to_make_randamData, Standardization_mean=False, Standardization_std=False).to(device)
        random_mask = torch.randint(0, 2, (batch_size,timeSeries_size), dtype=torch.float32).to(device)  #値が0or1のtorch.Size([batch_size, timeSeries_size])を作成
        #print(random_latent)
        #print(random_features)
        #print(random_mask)

        #値を変換
        #random_features = random_features*std_features+mean_features
        
        #最適化対象のすべてのパラメータの勾配を0にする
        optimizer_netD.zero_grad() 
        
        #"本物データ"を識別器でfp 
        real_validity = netD(timeSeries_input=real_timeSeries, params_input=real_params, features_input=real_features, mask_input=real_mask) #=torch.Size([batch_size, 1])
        
        #"偽データ"を識別器でfp
        fake_params, fake_timeSeries = netG(latent_input=random_latent, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, timeSeries_size]), torch.Size([batch_size, param_size])
        fake_validity = netD(timeSeries_input=fake_timeSeries, params_input=fake_params, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, 1])
        
        #まとめてlossを計算(本物は大きく, 偽物は小さく なってほしい)
        loss_netD = 1 - torch.mean(real_validity) + torch.mean(fake_validity) - 0 #これが小さくなってほしい
        #loss_netD = F.relu(1.0 - real_validity).mean() + F.relu(1.0 + fake_validity).mean() #Hinge Loss 多分ダメダメ
        #loss_netD = torch.mean(F.relu(0.2 - (real_validity - fake_validity))) #Hinge Loss
        real_validity_netD = round(torch.mean(real_validity).item(),30)
        fake_validity_netD = round(torch.mean(fake_validity).item(),30)
        #real_validity_netD = round(torch.mean(torch.sigmoid(real_validity)).item(),30)
        #fake_validity_netD = round(torch.mean(torch.sigmoid(fake_validity)).item(),30)
        
        #bp(=各パラメータの偏微分を計算)
        loss_netD.backward()
        #最適化
        optimizer_netD.step()
        
        
        #---------------------------------------------------------------------------------------------------------
        #「生成器の学習」
        
        #識別器の学習の何回おきに生成器を学習させるか(イテレーション数で指定)
        for n in range(netG_learn_n_start):
        #for n in range(netG_learn_n_start + int((netG_learn_n_end-netG_learn_n_start)*epoch/EPOCHS)):
            #[random data]生成器入力用データ
            random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            random_features = torch.randn((batch_size,features_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            #random_features = return_randam_data(batch_size, list_to_make_randamData, Standardization_mean=False, Standardization_std=False).to(device)
            random_mask = torch.randint(0, 2, (batch_size,timeSeries_size), dtype=torch.float32).to(device)  #値が0or1のtorch.Size([batch_size, timeSeries_size])を作成

            #値を変換
            #random_features = random_features*std_features+mean_features

            #最適化対象のすべてのパラメータの勾配を0にする
            optimizer_netG.zero_grad() 

            #"偽データ"を識別器でfp
            fake_params, fake_timeSeries = netG(latent_input=random_latent, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, timeSeries_size]), torch.Size([batch_size, param_size])
            fake_validity = netD(timeSeries_input=fake_timeSeries, params_input=fake_params, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, 1])
            
            #lossを計算(「本物は大きく, 偽物は小さく なってほしい」の逆)(=fake_validityは大きくなってほしい)
            loss_netG = -torch.mean(fake_validity) #これが小さくなってほしい #=torch.Size([1])
            fake_validity_netG = round(torch.mean(fake_validity).item(),30)
            #fake_validity_netG = round(torch.mean(torch.sigmoid(fake_validity)).item(),30)

            #---------------------------------------------------------------------------------------------------------
            #「パラメータに合うように、時系列を学習」
            #100エポック以上になったら「生成したparamsから生成した時系列データ」と「ganから生成した時系列データ」とのlossをloss_netGに追加
            if epoch >= m_i_GAN_start: #100エポック以上になったら
                #取り出す
                #fake_params_detached = fake_params.detach().to("cpu").numpy() #値は貰って共有、計算グラフは切る #& ndarry化
                fake_params_detached = fake_params.detach() #値は貰って共有、計算グラフは切る #& ndarry化
                random_features_detached = random_features.detach() #値は貰って共有、計算グラフは切る
                fake_timeSeries_detached = fake_timeSeries.detach() #値は貰って共有、計算グラフは切る
                #fake_validity_detached = fake_validity.detach() #値は貰って共有、計算グラフは切る
                #値を戻す
                fake_params_detached = fake_params_detached*std_params+mean_params
                random_features_detached = random_features_detached*std_features+mean_features
                fake_timeSeries_detached = fake_timeSeries_detached*std_timeSeries+mean_timeSeries
                #m-i-lossに利用するparams
                params_to_use = torch.cat((fake_params_detached,random_features_detached[:,0:1]),1).to("cpu").numpy()
                #m_i_lossの影響度
                degreeOfInfluence = 1/1
                #degreeOfInfluence = torch.abs(fake_validity_detached)/10  #多分変
                #m_i_lossを計算
                m_i_loss_raw = func_m_i_loss(params_to_use, fake_timeSeries_detached)
                #m_i_loss = torch.mean(m_i_loss_raw) #=torch.Size([batch_size,1])  #fake_validity_detached大ならfunc_m_i_loss小,小なら大
                m_i_loss = torch.mean(degreeOfInfluence*torch.sigmoid(m_i_loss_raw)) #=torch.Size([batch_size,1])  #fake_validity_detached大ならfunc_m_i_loss小,小なら大
                #print(m_i_loss_raw)
                #print(m_i_loss)
                #lossを追加
                loss_netG += m_i_loss #追加
                #lossをprint
                #print('m_i_loss = ' + str(m_i_loss))
            else:
                m_i_loss = torch.tensor(np.nan)
            
            #bp(=偏微分を計算)
            loss_netG.backward()
            #最適化
            optimizer_netG.step()

        #---------------------------------------------------------------------------------------------------------
        #print
        if epoch < m_i_GAN_start:
            print("iter:{5} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:None".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, iter_count))
            #print('[Epoch %d/%d] [loss_netD: %f] [loss_netG: %f]'% (epoch, EPOCHS, loss_netD.item(), loss_netG.item()))
        elif m_i_GAN_start <= epoch:
            print("iter:{6} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:{5}".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, round(m_i_loss.item(),30), iter_count))
            #print("iter:{5} epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:None".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, iter_count))

        #---------------------------------------------------------------------------------------------------------
        #sum
        loss_real_validity_sum += real_validity_netD
        loss_fake_validity_sum += fake_validity_netD
        loss_m_i_loss_sum += m_i_loss
        
        #---------------------------------------------------------------------------------------------------------
        #イテレーション数を+1
        iter_count += 1
        #print(iter_count)

    ############################################################################
    #「今エポックの最後のイテレーションのloss」を「loss履歴」に追加
    #losses_real_validity.append(real_validity_netD) 
    #losses_fake_validity.append(fake_validity_netD)
    #losses_m_i_loss.append(round(m_i_loss.item(),30))

    losses_real_validity.append(loss_real_validity_sum/(iter_count-1)) 
    losses_fake_validity.append(loss_fake_validity_sum/(iter_count-1))
    losses_m_i_loss.append(loss_m_i_loss_sum/(iter_count-1))
    
    ############################################################################
    #[評価]
    #---------------------------------------------------------------------------------------------------------
    #指定エポック毎に、「エポック数」,「識別機のloss」,「生成器のloss」を表示
    if epoch%print_loss_interval == 0:
        if epoch < m_i_GAN_start:
            print("epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:None".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG))
            #print('[Epoch %d/%d] [loss_netD: %f] [loss_netG: %f]'% (epoch, EPOCHS, loss_netD.item(), loss_netG.item()))
        elif m_i_GAN_start <= epoch:
            print("epoch:{0}/{1} loss_D_real:{2} loss_D_fake:{3} loss_G_fake:{4} m-i-loss:{5}".format(epoch, EPOCHS, real_validity_netD, fake_validity_netD, fake_validity_netG, round(m_i_loss.item(),30)))
    #---------------------------------------------------------------------------------------------------------
    #指定エポック毎に、生成データをplot
    if epoch%plot_fake_datas_interval == 0:
        #matplotlibのfig作成
        fig = plt.figure()
        #time作成
        t = np.arange(0, timeSeries_size, 1)
        #生成時系列データをplot
        for j in range(math.ceil(plot_fake_datas_n/batch_size)):
            #-----------------------------------
            #[random data]生成器入力用データ
            random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
            random_features = return_randam_data(batch_size, list_to_make_randamData, Standardization_mean=False, Standardization_std=False).to(device)
            random_mask = torch.randint(0, 2, (batch_size,timeSeries_size), dtype=torch.float32).to(device)  #値が0or1のtorch.Size([batch_size, timeSeries_size])を作成
            #値を変換
            random_features = random_features*std_features+mean_features
            #tauのみ変更
            random_features[:, 0] = 100
            #-----------------------------------
            #"生成器入力用ランダムデータ"を生成器でfp
            fake_params, fake_timeSeries = netG(latent_input=random_latent, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, timeSeries_size]), torch.Size([batch_size, param_size])
            #値を戻す
            fake_timeSeries = fake_timeSeries*std_timeSeries+mean_timeSeries
            #-----------------------------------
            #matplotlibで使うためにndarray化
            fake_timeSeries_numpy = fake_timeSeries.detach().to("cpu").numpy().copy() #detach() & ndarray化
            #時系列データ部分をplot
            for b in range(batch_size):
                plt.plot(t, fake_timeSeries_numpy[b], c='maroon', linewidth=0.2)
        #figの見た目設定
        plt.xlabel('day') #x軸ラベル
        plt.ylabel('antibody titer') #y軸ラベル
        plt.title('fake data (z='+str(latent_size)+', %d epochs)' % epoch) #タイトル
        plt.xticks(np.arange(0, 365, 30))
        #pauseする
        plt.pause(.01)
        #図を消す
        plt.close()
    #---------------------------------------------------------------------------------------------------------
    #指定エポック毎に、loss履歴をplot
    if epoch%plot_losses_interval == 0:
        #matplotlibのfig作成
        fig2 = plt.figure()
        #realのloss履歴をプロット
        plt.plot(range(len(losses_real_validity)), losses_real_validity, label='real_loss')
        #fakeのloss履歴をプロット
        plt.plot(range(len(losses_fake_validity)), losses_fake_validity, label='fake_loss')
        #figの見た目設定
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.ylim(0,1)
        #plt.yticks(np.arange(0, 1, 0.1))
        plt.legend()
        plt.grid()
        #表示
        plt.show()
        #pauseする
        plt.pause(.01)
        #図を消す
        plt.close()
    #---------------------------------------------------------------------------------------------------------
    """
    #modelをsave
    if epoch%save_models == 0:
        #make directory
        dir_path = "/Users/tatematsudaiki/Desktop/VPGAM_models"
        os.makedirs(dir_path,exist_ok=True)
        #save models
        torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(dir_path,"netG",epoch))
        torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(dir_path,"netD",epoch))
    """"""

In [ ]:
"""
#epoch
epoch_write = 26
lr_write = "001"

#save model
model_path = "/Users/tatematsudaiki/Desktop"
torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(model_path,"netG",epoch_write,lr_write))
torch.save(netG.to('cpu').state_dict(), "{0}/{1}_epoch{2}.pth".format(model_path,"netD",epoch_write,lr_write))
"""

In [ ]:
"""
#load model
model_path = "/Users/tatematsudaiki/Desktop"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
"""

In [ ]:
############################################################################
#[評価]

In [ ]:
"""#matplotlibのfig作成
fig = plt.figure()
#time作成
t = np.arange(0, timeSeries_size, 1)
#生成時系列データをplot
for j in range(math.ceil(plot_fake_datas_n/batch_size)):
    #[random data]生成器入力用データ
    random_latent = torch.randn((batch_size,latent_size), dtype=torch.float32).to(device) #=torch.Size([batch_size, latent_size])
    random_features = return_randam_data(batch_size, list_to_make_randamData)
    random_features[:, 0] = 30
    random_mask = torch.randint(0, 2, (batch_size,timeSeries_size), dtype=torch.float32).to(device)  #値が0or1のtorch.Size([batch_size, timeSeries_size])を作成
    
    #"生成器入力用ランダムデータ"を生成器でfp
    fake_params, fake_timeSeries = netG(latent_input=random_latent, features_input=random_features, mask_input=random_mask) #=torch.Size([batch_size, timeSeries_size]), torch.Size([batch_size, param_size])
    fake_timeSeries_numpy = fake_timeSeries.detach().to("cpu").numpy().copy() #detach() & ndarray化

    #時系列データ部分をプロット
    for b in range(batch_size):
        plt.plot(t, fake_timeSeries_numpy[b], c='maroon', linewidth=0.2)
        
#figの見た目設定
plt.xlabel('time') #x軸ラベル
plt.ylabel('viralload') #y軸ラベル
plt.title('fake data (z='+str(latent_size)+', %d epochs)' % epoch) #タイトル
plt.xticks(np.arange(0, 365, 30))"""

In [ ]:
"""#loss履歴を表示

#matplotlibのfig作成
fig2 = plt.figure()
#Discriminator(識別器)のloss履歴をプロット
plt.plot(range(len(losses_netD)), losses_netD, label='Discriminator_Loss')
#Generator(生成器)のloss履歴をプロット
plt.plot(range(len(losses_netG)), losses_netG, label='Generator_Loss')
#figの見た目設定
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.grid()
#保存
#fig2.savefig('results(dropped_data)/loss.png')
#表示
plt.show()"""

# test

In [ ]:
"""ID = 2000

#read
df_params = pd.read_csv("../params_vaccination2_withoutgroup0.csv",sep=',')\
    .query('id==@ID')

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html
from scipy.integrate import solve_ivp
#paramsを取り出す(V0,β,γ,δ)
M10_M20_A0 = [100,100,0] #M10(=D1),M20(=D2)はテキトーに決められている
H2 = df_params["H2"].item()
m = df_params["m"].item()
tau1 = df_params["tau1"].item()
tau2 = df_params["tau2"].item()
print(H2)
print(m)
print(tau1)
print(tau2)
#constants (estimated by 12 health care workers)
d = 0.693
mu = 0.885346
H1 = 975.297
eta1 = 12.52
K = 33900
eta2 = 4.25

init   = M10_M20_A0
t_span = [0.0,365.0]
t_eval = np.linspace(*t_span,1000) # time for sampling

#常微分方程式を定義 / vaccination2 ver
def func(t,M1_M2_A, H2, m, tau1, tau2, d, mu, H1, eta1, K, eta2):
    #print(t)
    #extract
    M1,M2,A = M1_M2_A
    #formulas
    dM1dt = -d*M1 if tau1<=t else 0
    dM2dt = -d*M2 if tau2<=t else 0
    if 0<=t<tau1+eta1:
        dAdt = 0
    elif tau1+eta1<=t<tau2+eta2:
        dAdt = H1*(M1**m/(K**m+M1**m)) - mu*A
    elif tau2+eta2<=t:
        dAdt = H2*((M1+M2)**m/(K**m+(M1+M2)**m)) - mu*A

    #return
    return [dM1dt, dM2dt, dAdt]

sol = solve_ivp(fun=func,
                t_span=t_span,
                y0=init,
                method='RK23',
                t_eval=t_eval,
                args=(H2, m, tau1, tau2, d, mu, H1, eta1, K, eta2))

print(sol.t.size)
#print(sol.t)
print(sol.y.shape)
#print(sol.y)
plt.plot(sol.t,sol.y[2])


#plot
###################################################
#read
df_raw = pd.read_csv("../timeSeries_raw_vaccination2_withoutgroup0.csv",sep=',',encoding="shift-jis", index_col=0, header=0)
df_fitted = pd.read_csv("../timeSeries_fitted_vaccination2_withoutgroup0.csv",sep=',',encoding="shift-jis", index_col=0, header=0)
#----------------------------------------------------------------
#df_raw
#----------------------------------------------------------------
#times
times = df_raw.columns.values
#extract
values = df_raw.loc[ID].values
#plot
plt.plot(times, values, marker="o", color="red", markersize=5, linewidth=1, zorder=2, linestyle="None")
#----------------------------------------------------------------
#df_fitted
#----------------------------------------------------------------
#times
times = df_fitted.columns.values
#extract
values = df_fitted.loc[ID].values
#plot
plt.plot(times, values, marker="o", color="black", markersize=2, linewidth=1, zorder=1)"""

In [ ]:
#使わなかった部分(preprocessing.py)
"""def compute_real_MSE(fake_data, real_data, mask_table, index):
    mask = mask_table[index]
    masked_fake_data = fake_data * mask
    return ((real_data - masked_fake_data) ** 2).mean()


def calculate_sim_realdata(params):
    v0 = [1, params[3]]
    beta = params[0]
    gamma = params[1]
    delta = params[2]
    t = np.arange(0, 31, 0.01)
    simulated_data = odeint(func, v0, t, args=(beta, gamma, delta))

    processed_data = simulated_data[::100]
    processed_data = np.log10(processed_data[:, 1])
    sim_realdata = np.where(processed_data >= 2, processed_data, 2)
    return sim_realdata"""

In [ ]:
#使わなかった部分(grud.py)
# GPUじゃないからいらないコード
    '''
        if not any_param.is_cuda or not torch.backends.cudnn.is_acceptable(any_param):
            return

        # If any parameters alias, we fall back to the slower, copying code path. This is
        # a sufficient check, because overlapping parameter buffers that don't completely
        # alias would break the assumptions of the uniqueness check in
        # Module.named_parameters().
        all_weights = self._flat_weights
        unique_data_ptrs = set(p.data_ptr() for p in all_weights)
        if len(unique_data_ptrs) != len(all_weights):
            return

        with torch.cuda.device_of(any_param):
            import torch.backends.cudnn.rnn as rnn

            # NB: This is a temporary hack while we still don't have Tensor
            # bindings for ATen functions
            with torch.no_grad():
                # NB: this is an INPLACE function on all_weights, that's why the
                # no_grad() is necessary.
                torch._cudnn_rnn_flatten_weight(
                    all_weights, (4 if self.bias else 2),
                    self.input_size, rnn.get_cudnn_mode(self.mode), self.hidden_size, self.num_layers,
                    self.batch_first, bool(self.bidirectional))
    '''

    """def _apply(self, fn):
        ret = super(GRUD, self)._apply(fn)
        self.flatten_parameters()
        return ret"""

    """# 次元数とか合ってるか確認するメソッド
    def check_forward_args(self, input, hidden, batch_sizes):
        is_input_packed = batch_sizes is not None
        expected_input_size = 2 if is_input_packed else 3
        if input.dim() != expected_input_size:
            raise RuntimeError(
                'input must have {} dimensions, got {}'.format(
                    expected_input_size, input.dim()))
        if self.input_size != input.size(-1):
            raise RuntimeError(
                'input.size(-1) must be equal to input_size. Expected {}, got {}'.format(
                    self.input_size, input.size(-1)))

        if is_input_packed:
            mini_batch = int(batch_sizes[0])
        else:
            mini_batch = input.size(0) if self.batch_first else input.size(1)

        num_directions = 2 if self.bidirectional else 1
        expected_hidden_size = (self.num_layers * num_directions,
                                mini_batch, self.hidden_size)

        def check_hidden_size(hx, expected_hidden_size, msg='Expected hidden size {}, got {}'):
            if tuple(hx.size()) != expected_hidden_size:
                raise RuntimeError(msg.format(expected_hidden_size, tuple(hx.size())))

        if self.mode == 'LSTM':
            check_hidden_size(hidden[0], expected_hidden_size,
                              'Expected hidden[0] size {}, got {}')
            check_hidden_size(hidden[1], expected_hidden_size,
                              'Expected hidden[1] size {}, got {}')
        else:
            check_hidden_size(hidden, expected_hidden_size)"""

    """def extra_repr(self):
        s = '{input_size}, {hidden_size}'
        if self.num_layers != 1:
            s += ', num_layers={num_layers}'
        if self.bias is not True:
            s += ', bias={bias}'
        if self.batch_first is not False:
            s += ', batch_first={batch_first}'
        if self.dropout != 0:
            s += ', dropout={dropout}'
        if self.bidirectional is not False:
            s += ', bidirectional={bidirectional}'
        return s.format(**self.__dict__)"""

    """def __setstate__(self, d):
        super(GRUD, self).__setstate__(d)
        if 'all_weights' in d:
            self._all_weights = d['all_weights']
        if isinstance(self._all_weights[0][0], str):
            return
        num_layers = self.num_layers
        num_directions = 2 if self.bidirectional else 1
        self._all_weights = []

        weights = ['weight_dg_x', 'weight_dg_h',
                   'weight_xz', 'weight_hz', 'weight_mz',
                   'weight_xr', 'weight_hr', 'weight_mr',
                   'weight_xh', 'weight_hh', 'weight_mh',
                   'weight_hy',
                   'bias_dg_x', 'bias_dg_h',
                   'bias_z', 'bias_r', 'bias_h', 'bias_y']

        if self.bias:
            self._all_weights += [weights]
        else:
            self._all_weights += [weights[:2]]"""

    """@property
    def _flat_weights(self):
        return list(self._parameters.values())"""

    """@property
    def all_weights(self):
        return [[getattr(self, weight) for weight in weights] for weights in self._all_weights]"""